Generate high performance C++ code from Python code

In [ ]:
import os
from dotenv import load_dotenv
import gradio as gr
from openai import OpenAI
from IPython.display import display, Markdown
from system_info import retrieve_system_info
import io
import sys
import subprocess

In [ ]:
load_dotenv(override=True)

GROQ_BASE_URL = "https://api.groq.com/openai/v1"
OLLAMA_BASE_URL = "http://localhost:11434/v1"

groq_api_key = os.getenv('GROQ_API_KEY')

if groq_api_key:
    print(f"Grok API Key exists and begins {groq_api_key[:4]}")
else:
    print("Grok API Key not set (and this is optional)")

groq = OpenAI(base_url=GROQ_BASE_URL, api_key=groq_api_key)
ollama = OpenAI(base_url=OLLAMA_BASE_URL, api_key="ollama")

In [ ]:
models = ["openai/gpt-oss-120b", "deepseek-coder-v2", "openai/gpt-oss-20b", "qwen2.5-coder:1.5b"]

clients = {"openai/gpt-oss-120b": groq, "deepseek-coder-v2": ollama, 
           "openai/gpt-oss-20b":  groq, "qwen2.5-coder:1.5b": ollama}

In [ ]:
system_info = retrieve_system_info()

In [ ]:
message = f"""
Here is a report of the system information for my computer.
I want to run a C++ compiler to compile a single C++ file called main.cpp and then execute it in the simplest way possible.
Please reply with whether I need to install any C++ compiler to do this. If so, please provide the simplest step by step instructions to do so.


If I'm already set up to compile C++ code, then I'd like to run something like this in Python to compile and execute the code:
```python
compile_command = # something here - to achieve the fastest possible runtime performance
compile_result = subprocess.run(compile_command, check=True, text=True, capture_output=True)
run_command = # something here
run_result = subprocess.run(run_command, check=True, text=True, capture_output=True)
return run_result.stdout
```
Please tell me exactly what I should use for the compile_command and run_command.
If I already have all requirements then just give compile_command and run_command only.

System information:
{system_info}
"""

response = groq.chat.completions.create(model='openai/gpt-oss-120b', messages=[{"role": "user", "content": message}])
display(Markdown(response.choices[0].message.content))
    

In [ ]:
compile_command = ["g++","-O3","-march=native","-flto","-std=c++20","main.cpp","-o",
                   "main.exe"]
run_command = ["main.exe"]

In [ ]:
system_prompt = """
Your task is to convert Python code into high performance C++ code.
Respond only with C++ code. Do not provide any explanation other than occasional comments.
The C++ response needs to produce an identical output in the fastest possible time.
"""

def user_prompt_for(python):
    return f"""
Port this Python code to C++ with the fastest possible implementation that produces identical output in the least time.
The system information is:
{system_info}
Your response will be written to a file called main.cpp and then compiled and executed; the compilation command is:
{compile_command}
Respond only with C++ code.
Python code to port:

```python
{python}
```
"""

In [ ]:
def messages_for(python):
    return [{"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt_for(python)}]

In [ ]:
def write_output(cpp):
    with open("main.cpp", "w") as f:
        f.write(cpp)

In [ ]:
def port(model, python):
    client = clients[model]
    reasoning_effort = "high" if "gpt" in model else None
    response = client.chat.completions.create(model=model, messages=messages_for(python), reasoning_effort=reasoning_effort)
    reply = response.choices[0].message.content
    write_output(reply)
    return reply

In [ ]:
pi = """
import time

def calculate(iterations, param1, param2):
    result = 1.0
    for i in range(1, iterations+1):
        j = i * param1 - param2
        result -= (1/j)
        j = i * param1 + param2
        result += (1/j)
    return result

start_time = time.time()
result = calculate(200_000_000, 4, 1) * 4
end_time = time.time()

print(f"Result: {result:.12f}")
print(f"Execution Time: {(end_time - start_time):.6f} seconds")
"""

In [ ]:
def run_python(code):
    globals_dict = {"__builtins__": __builtins__}

    buffer = io.StringIO()
    old_stdout = sys.stdout
    sys.stdout = buffer

    try:
        exec(code, globals_dict)
        output = buffer.getvalue()
    except Exception as e:
        output = f"Error: {e}"
    finally:
        sys.stdout = old_stdout

    return output

In [ ]:
print(run_python(pi))

In [ ]:
def compile_and_run():
    try:
        subprocess.run(compile_command, check=True, text=True, capture_output=True)
        print(subprocess.run(run_command, check=True, text=True, capture_output=True).stdout)
        print(subprocess.run(run_command, check=True, text=True, capture_output=True).stdout)
        print(subprocess.run(run_command, check=True, text=True, capture_output=True).stdout)
    except subprocess.CalledProcessError as e:
        print(f"An error occurred:\n{e.stderr}")

In [ ]:
with gr.Blocks(title="Python To CPP") as ui:
    with gr.Row():
        python = gr.Textbox(label="Python code:", lines=28, value=pi)
        cpp = gr.Textbox(label="C++ code:", lines=28)
    with gr.Row():
        model = gr.Dropdown(label="Select model", choices=models, value=models[0])
        convert = gr.Button("Convert Code")
    
    convert.click(fn=port, inputs=[model, python], outputs=[cpp])

ui.launch(inbrowser=True)


In [ ]:
from styles import CSS

with gr.Blocks(css=CSS, theme=gr.themes.Monochrome(), title=f"Port from Python to CPP") as ui:
    with gr.Row(equal_height=True):
        with gr.Column(scale=6):
            python = gr.Code(
                label="Python (original)",
                value=pi,
                language="python",
                lines=26
            )
        with gr.Column(scale=6):
            cpp = gr.Code(
                label=f"C++ (generated)",
                value="",
                language="cpp",
                lines=26
            )

    with gr.Row(elem_classes=["controls"]):
        python_run = gr.Button("Run Python", elem_classes=["run-btn", "py"])
        model = gr.Dropdown(models, value=models[0], show_label=False)
        convert = gr.Button(f"Port to CPP", elem_classes=["convert-btn"])
        cpp_run = gr.Button(f"Run CPP", elem_classes=["run-btn", "cpp"])

    with gr.Row(equal_height=True):
        with gr.Column(scale=6):
            python_out = gr.TextArea(label="Python result", lines=8, elem_classes=["py-out"])
        with gr.Column(scale=6):
            cpp_out = gr.TextArea(label=f"CPP result", lines=8, elem_classes=["cpp-out"])

    convert.click(fn=port, inputs=[model, python], outputs=[cpp])
    python_run.click(fn=run_python, inputs=[python], outputs=[python_out])
    cpp_run.click(fn=compile_and_run, inputs=[cpp], outputs=[cpp_out])

ui.launch(inbrowser=True)


In [ ]:
compile_and_run()